In [ ]:
!pip install azure-ai-formrecognizer

In [2]:
#import libraries
import os
from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

In [3]:
load_dotenv()

True

In [4]:
endpoint = os.getenv("ENDPOINT")
key = os.getenv("KEY")

In [6]:
fileruri = "https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/blob/main/Labfiles/prebuilt-doc-intelligence/sample-invoice/sample-invoice.pdf?raw=true""01. DocumentIntelligence_SDK.ipynb"
fileModel = "prebuilt-invoice"

In [7]:
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

In [8]:
poller = document_analysis_client.begin_analyze_document_from_url(
    model_id=fileModel,
    document_url=fileruri
)

In [11]:
results = poller.result()

In [17]:
result = results.documents[0]

In [18]:
result.fields

{'AmountDue': DocumentField(value_type=currency, value=CurrencyValue(amount=610.0, symbol=$, code=USD), content=$610.00, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.3587, y=7.8012), Point(x=7.9174, y=7.8012), Point(x=7.9222, y=7.9731), Point(x=7.3539, y=7.9731)])], spans=[DocumentSpan(offset=653, length=7)], confidence=0.949),
 'BillingAddress': DocumentField(value_type=address, value=AddressValue(house_number=123, po_box=None, road=Bill St, city=Redmond, state=WA, postal_code=98052, country_region=None, street_address=123 Bill St, unit=None, city_district=None, state_district=None, suburb=None, house=None, level=None), content=123 Bill St,
 Redmond WA, 98052, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.5635, y=4.3589), Point(x=2.0247, y=4.3589), Point(x=2.0247, y=4.7266), Point(x=0.5635, y=4.7266)])], spans=[DocumentSpan(offset=328, length=12), DocumentSpan(offset=370, length=17)], confidence=0.889),
 'BillingAddressRecipient': DocumentFie

In [22]:
for k, v in result.fields.items():
    if v.value_type == 'address' and v.value:
        print(f"{k}: {v.value.street_address}, {v.value.city}, {v.value.state}, {v.value.postal_code} (confidence: {v.confidence})")
    else:
        print(f"{k}: {v.value} (confidence: {v.confidence})")

AmountDue: $610.0 (confidence: 0.949)
BillingAddress: 123 Bill St, Redmond, WA, 98052 (confidence: 0.889)
BillingAddressRecipient: Microsoft Finance (confidence: 0.938)
CustomerAddress: 123 Other St, Redmond, WA, 98052 (confidence: 0.888)
CustomerAddressRecipient: Microsoft Corp (confidence: 0.936)
CustomerId: CID-12345 (confidence: 0.967)
CustomerName: MICROSOFT CORPORATION (confidence: 0.919)
DueDate: 2019-12-15 (confidence: 0.971)
InvoiceDate: 2019-11-15 (confidence: 0.971)
InvoiceId: INV-100 (confidence: 0.971)
InvoiceTotal: $110.0 (confidence: 0.969)
Items: [DocumentField(value_type=dictionary, value={'Amount': DocumentField(value_type=currency, value=CurrencyValue(amount=100.0, symbol=$, code=USD), content=$100.00, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.3635, y=6.0729), Point(x=7.9126, y=6.0729), Point(x=7.9126, y=6.2305), Point(x=7.3635, y=6.2352)])], spans=[DocumentSpan(offset=562, length=7)], confidence=0.913), 'Description': DocumentField(value_typ

In [21]:
result.fields.get("AmountDue")

DocumentField(value_type=currency, value=CurrencyValue(amount=610.0, symbol=$, code=USD), content=$610.00, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.3587, y=7.8012), Point(x=7.9174, y=7.8012), Point(x=7.9222, y=7.9731), Point(x=7.3539, y=7.9731)])], spans=[DocumentSpan(offset=653, length=7)], confidence=0.949)

#### Using the NEW SDK for Document Intelligence

In [ ]:
!pip install azure-ai-documentintelligence

In [12]:
#import libraries
import os
from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import DocumentAnalysisFeature, AnalyzeResult

In [2]:
load_dotenv()

True

In [3]:
endpoint = os.getenv("ENDPOINT")
key = os.getenv("KEY")

In [11]:
document_intelligence_client = DocumentIntelligenceClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

In [9]:
def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (word.span.offset + word.span.length) <= (span.offset + span.length):
            return True
    return False

def _format_bounding_region(bounding_regions):
    if not bounding_regions:
        return "N/A"
    return ", ".join(
        f"Page #{region.page_number}: {_format_polygon(region.polygon)}" for region in bounding_regions
    )

def _format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join([f"[{polygon[i]}, {polygon[i + 1]}]" for i in range(0, len(polygon), 2)])


In [10]:
print(os.getcwd())

c:\Users\JordanMiller\Documents\_StormSurge\06162025-Microsoft-AI-Solutions\06. Knowledge Mining


In [23]:
with open("data/invoice-1235.pdf", "rb") as f:
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",
        f,
        features=[DocumentAnalysisFeature.KEY_VALUE_PAIRS],
        content_type="application/octet-stream",
    )

In [24]:
result = poller.result()

In [16]:
result

{'apiVersion': '2024-11-30', 'modelId': 'prebuilt-layout', 'stringIndexType': 'textElements', 'content': 'Contoso Ltd 2 Main St, Bigtown, England, EH1 234 Tel: 555 123-4567\nInvoice No:\n1234\nDate:\n03/07/2025\nCustomer Name:\nJohn Smith 123 River Street\nAddress:\nMarshtown\nEngland\nGL1 234\nItem\nPrice\nQuantity\nItem Total\n38mm Widget\n24.50\n2\n49.00\n3.5mm screws pack\n4.99\n1\n4.99\nLeft-handed screwdriver\n7.49\n1\n7.49\nSubtotal\n61.48\nTax\n6.14\nShipping\n15.00\nTotal Due\n82.62', 'pages': [{'pageNumber': 1, 'angle': 0.21717900037765503, 'width': 8.5, 'height': 11, 'unit': 'inch', 'words': [{'content': 'Contoso', 'polygon': [0.8658, 0.8219, 1.8756, 0.8213, 1.8755, 1.0372, 0.8656, 1.04], 'confidence': 0.997, 'span': {'offset': 0, 'length': 7}}, {'content': 'Ltd', 'polygon': [1.9609, 0.8204, 2.3298, 0.8152, 2.3298, 1.039, 1.9608, 1.0369], 'confidence': 0.995, 'span': {'offset': 8, 'length': 3}}, {'content': '2', 'polygon': [0.8578, 1.0917, 0.9242, 1.0916, 0.9239, 1.2566, 0.8

In [25]:
if result.styles:
    for style in result.styles:
        if style.is_handwritten:
            print("Document contains handwritten content: ")
            print(",".join([result.content[span.offset : span.offset + span.length] for span in style.spans]))


Document contains handwritten content: 
Jordan WAS here


In [ ]:
if result.key_value_pairs:
    for kv_pair in result.key_value_pairs:
        if kv_pair.key:
            print(
                f"Key '{kv_pair.key.content}' found within "
                f"'{_format_bounding_region(kv_pair.key.bounding_regions)}' bounding regions"
            )
        if kv_pair.value:
            print(
                f"Value '{kv_pair.value.content}' found within "
                f"'{_format_bounding_region(kv_pair.value.bounding_regions)}' bounding regions\n"
            )

In [ ]:
if result.tables:
    for table_idx, table in enumerate(result.tables):
        print(f"Table # {table_idx} has {table.row_count} rows and {table.column_count} columns")
        if table.bounding_regions:
            for region in table.bounding_regions:
                print(
                    f"Table # {table_idx} location on page: {region.page_number} is {_format_polygon(region.polygon)}"
                )
        for cell in table.cells:
            print(f"...Cell[{cell.row_index}][{cell.column_index}] has text '{cell.content}'")
            if cell.bounding_regions:
                for region in cell.bounding_regions:
                    print(
                        f"...content on page {region.page_number} is within bounding polygon '{_format_polygon(region.polygon)}'\n"
                    )